In [2]:
using Transformers

In [4]:
function load_model(; load_head = true, kwrgs...)
    model_name = "karoldobiczek/relitc-FOMC-CMLM"
    tkr = Transformers.load_tokenizer(model_name)
    cfg = Transformers.HuggingFace.HGFConfig(Transformers.load_config(model_name); kwrgs...)
    mod = Transformers.load_model(model_name, "ForMaskedLM"; config = cfg)

    return tkr, mod, cfg
end

load_model (generic function with 1 method)

In [6]:
tkr, model = load_model()

(BertTextEncoder(
├─ TextTokenizer(MatchTokenization(WordPieceTokenization(bert_uncased_tokenizer, WordPiece(vocab_size = 30522, unk = [UNK], max_char = 100)), 5 patterns)),
├─ vocab = Vocab{String, SizedArray}(size = 30522, unk = [UNK], unki = 101),
├─ startsym = [CLS],
├─ endsym = [SEP],
├─ padsym = [PAD],
├─ trunc = 512,
└─ process = Pipelines:
  ╰─ target[token] := TextEncodeBase.nestedcall(string_getvalue, source)
  ╰─ target[token] := Transformers.TextEncoders.grouping_sentence(target.token)
  ╰─ target[(token, segment)] := SequenceTemplate{String}([CLS]:<type=1> Input[1]:<type=1> [SEP]:<type=1> (Input[2]:<type=2> [SEP]:<type=2>)...)(target.token)
  ╰─ target[attention_mask] := (NeuralAttentionlib.LengthMask ∘ Transformers.TextEncoders.getlengths(512))(target.token)
  ╰─ target[token] := TextEncodeBase.trunc_and_pad(512, [PAD], head, tail)(target.token)
  ╰─ target[token] := TextEncodeBase.nested2batch(target.token)
  ╰─ target[segment] := TextEncodeBase.trunc_and_pad(512, 1, hea

In [12]:
input = TextEncoders.encode(tkr, "[SEP] hello world!")

(token = Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], segment = [1, 1, 1, 1, 1, 1], attention_mask = NeuralAttentionlib.LengthMask{1, Vector{Int32}}(Int32[6]))

In [13]:
out = model(input)

(hidden_state = Float32[-0.0068912334 1.0873084 … -0.60200036 1.0715828; 0.12056827 0.10844539 … -0.6849348 0.116397835; … ; -0.026128935 -0.5781372 … -0.010042516 -0.5654973; 0.13880502 -0.25988412 … -0.09458274 -0.27784675;;;], attention_mask = NeuralAttentionlib.LengthMask{1, Vector{Int32}}(Int32[6]), logit = Float32[-6.8121076 -13.4576 … -11.932923 -13.186207; -6.7392855 -13.34565 … -12.13822 -13.120222; … ; -6.0473905 -10.81515 … -10.769718 -10.597319; -4.2169976 -12.677718 … -3.9266496 -12.219977;;;])

In [14]:
TextEncoders.decode(tkr, out.logit)

6×1 Matrix{String}:
 "."
 "."
 "hello"
 "world"
 "!"
 "."